<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Standard-Packages" data-toc-modified-id="Import-Standard-Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Standard Packages</a></span></li><li><span><a href="#Import-Symawofo-Packages" data-toc-modified-id="Import-Symawofo-Packages-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Symawofo Packages</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Add-Additional-Features" data-toc-modified-id="Add-Additional-Features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add Additional Features</a></span></li><li><span><a href="#Construct-Base-Model" data-toc-modified-id="Construct-Base-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Construct Base Model</a></span></li><li><span><a href="#Define-all-the-SubStrats" data-toc-modified-id="Define-all-the-SubStrats-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Define all the SubStrats</a></span></li><li><span><a href="#Train/Test-the-SubStrats" data-toc-modified-id="Train/Test-the-SubStrats-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Train/Test the SubStrats</a></span></li></ul></div>

# Import Standard Packages

In [ ]:
%load_ext autoreload 
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

from pathlib import Path
from copy import deepcopy
from datetime import datetime
import dill
from tqdm.auto import tqdm
import json

# internal
import sys
sys.path.append(r"/home/osindi/ami")
sys.path.append(r"/home/osindi/Symawofo")
sys.path.append(r"C:\SyMAWoFlo\Symawofo")
sys.path.append(r"C:\Users\amoniri\PycharmProjects\Symawofo")
sys.path.append(r"C:\Users\amoniri\PycharmProjects\ami")

# Import Symawofo Packages

In [ ]:
from symawofo import make_pipeline, WindowFrame, DataSplitter, FeatureFrame, FeatureSeries
from symawofo.transformers import Lag, Difference, StandardScaler, TransformedTargetRegressor, Clipper
from symawofo.model_selection.sk import GridSearchCV
from symawofo.models.base import model as model_dec, RegressorMixin2d, BaseModel
from symawofo.signals.wf import ThresholdSignSignal
from symawofo.pnl.base import AssetType
from symawofo.metrics import make_scorer
from symawofo.pnl import PNL
from symawofo.shared.utils import to_pickle, read_pickle

In [ ]:
from symawofo import get_client
client = get_client('local')

# Load Data

In [ ]:
df_master_daily = pd.read_csv('Transformed_Data.csv', index_col=0, parse_dates=True, infer_datetime_format=True)
df_master_daily = df_master_daily.join(pd.read_csv('DXY.csv', index_col=0))
df_master_daily['US_Equity_Real_Vol1m'] = np.exp(df_master_daily['US_Equity_Fut']).pct_change().rolling(21).std()*np.sqrt(252)
df_master_weekly = df_master_daily[df_master_daily.index.weekday==4].copy() # Convert to weekly! Fridays (i.e. .resample('W-FRI').last())
df_master_weekly.tail()

# Add Additional Features

In [ ]:
def zscore(x, w=10):
    return (x-x.rolling(w).mean())/x.rolling(w).std()


df_master_weekly['US_Equity_Fut_ChgP'] = np.exp(df_master_weekly['US_Equity_Fut']).pct_change()
df_master_weekly['US_Equity_Vol_Chg'] = df_master_weekly['US_Equity_Vol'].diff()
df_master_weekly['Mom_Change_SPX'] = df_master_weekly['US_Equity'].pct_change(4)*52/4 - df_master_weekly['US_Equity'].pct_change(52)
df_master_weekly['DXY_ChgP'] = df_master_weekly['DXY'].pct_change()
df_master_weekly['SPGSIN_Chg13'] = df_master_weekly['GLOB_Ind_metals'].diff(13)
df_master_weekly['eq_per_flows_USA'] = df_master_weekly['eq_per_flows_USA']
df_master_weekly['US_Equity_Real_Vol1m_Chg'] = df_master_weekly['US_Equity_Real_Vol1m'].diff()
df_master_weekly['US_Lois3m_Chg'] = df_master_weekly['US_Lois3m'].diff()
df_master_weekly['US_2y_v_OIS_ZSc52'] = zscore(df_master_weekly['US_2y_v_OIS'], 52)
df_master_weekly['US_Bond_30y_ZSc52'] = zscore(df_master_weekly['US_Bond_30y'], 52)
df_master_weekly['US_10y_2y_ZSc52'] = zscore((df_master_weekly['US_Bond_10y']-df_master_weekly['US_Bond_2y']), 52)
df_master_weekly['GLOB_EM_HCSp_Chg'] = df_master_weekly['GLOB_EM_HCSp'].diff()

In [ ]:
df_master_weekly['US_Equity_Real_Vol1m'].diff().plot()

# Construct Base Model

In [ ]:
from sklearn.model_selection import ParameterGrid

@model_dec(RegressorMixin2d)
class SubStrat(sk.base.BaseEstimator):
    def __init__(self, param_grid, scoring):
        self.param_grid = ParameterGrid(param_grid)
        
        self.func = None
        self.thresh = None
        
        self.best_params_ = {'func': None, 'thresh': None, 'score': -np.inf}
        
        self.scorer = scoring
        
    def fit(self, X, y):
        
        for params in self.param_grid:
            self.func = params['func']
            self.thresh = params['thresh']
            score = self.scorer(y, self.predict(X).data)
            if score > self.best_params_['score']:
                self.best_params_['func'] = self.func
                self.best_params_['thresh'] = self.thresh
                self.best_params_['score'] = score
                
        self.func = self.best_params_['func']
        self.thresh = self.best_params_['thresh']

        return self
    
    def predict(self, X):
        return self.func(X, self.thresh)

def make_model(param_grid, scoring_func):

    my_model = SubStrat(lags=0, param_grid=param_grid, scoring=scoring_func)
    
    return my_model
    
# This is a signal class compatible with Symawofo for 'Buy&Hold' strategy
class LongOnly:
    def __init__(self, target, index):
        self.target = target
        self.index = index

    def get_signals(self):
        return pd.Series(np.ones(len(target)), index=target.data.index, dtype='float').reindex(self.index)
    
    
def long_only_relative_mean_score(y, y_pred):
    return np.mean(y_pred*y) - np.mean((1-y_pred)*y)

def mean_score(y, y_pred):
    return np.mean(y_pred*y)

def sharpe_score(y, y_pred):
    return np.mean(y_pred*y)/np.std(y_pred*y)

def active_mean_score(y, y_pred):
    idx = np.abs(y_pred)>0
    return np.mean(y_pred[idx]*y[idx])

def active_sharpe_score(y, y_pred):
    idx = np.abs(y_pred)>0
    return np.mean(y_pred[idx]*y[idx])/np.std(y_pred[idx]*y[idx])

def ctail_score(y, y_pred):
    strat = y_pred*y
    top_percentile = np.percentile(strat, 90)
    idx = strat>top_percentile
    return np.mean(strat[idx])

def short(fn):
    def wrapper(x, y):
        return -fn(x, y)
    return wrapper


def mixed(x, threshold): #param_grid_func
    return np.less(x, threshold).astype(float)-np.greater(x, threshold).astype(float)

In [ ]:
df_master_weekly['US_Equity_Vol_Chg'].plot()

# Define all the SubStrats

In [ ]:
asset = 'US_Equity_Fut'


substrat_meta_list = [
 {'feature_name': 'US_Equity_Fut_ChgP',
  'param_grid': {'func': [np.less], 'thresh': [-0.005, -0.0025, 0.0, 0.0025, 0.005]},
    'feature_preprocessing': None,
  'scoring_func': mean_score},
                      
 {'feature_name': 'US_Equity_Real_Vol1m_Chg',
  'param_grid': {'func': [mixed], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'US_Lois3m_Chg',
  'param_grid': {'func': [mixed], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'US_Equity_Vol_Chg',
   'param_grid': {'func': [np.greater], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'DXY_ChgP',
  'param_grid': {'func': [np.less], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'SPGSIN_Chg13',
  'param_grid': {'func': [np.greater], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'eq_per_flows_USA',
  'param_grid': {'func': [np.less, np.greater], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'US_2y_v_OIS_ZSc52',
  'param_grid': {'func': [np.less], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'US_Bond_30y_ZSc52',
  'param_grid': {'func': [np.less], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'US_10y_2y_ZSc52',
  'param_grid': {'func': [np.less], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
                      
 {'feature_name': 'GLOB_EM_HCSp_Chg',
  'param_grid': {'func': [np.greater], 'thresh': np.linspace(-2, 2, num=20)},
 'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler()),
 'scoring_func': mean_score},
]



common_meta_dict = {
    'splitter': DataSplitter(test_size=13,
                             initial_training_size=104,
                             expanding=True,
                             last_training_window=True),
    'feature_preprocessing': None,
#     'feature_preprocessing': make_pipeline(StandardScaler()),
    'target_preprocessing': make_pipeline(Difference(), Lag(-1)),
    'scoring_func': lambda y, y_pred: np.mean(y_pred*y) - np.mean((1-y_pred)*y),
    'target_name': asset
}

# Add common meta data to each entry in substrat_meta_list
for k, v in common_meta_dict.items():
    for i in range(len(substrat_meta_list)):
        if k not in substrat_meta_list[i]:  # Only override if it does not already exist
            substrat_meta_list[i][k] = deepcopy(v)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, c

In [ ]:
for s in substrat_meta_list:
    s['scoring_func'] = long_only_relative_mean_score

In [ ]:

# spx_change_dict = {'feature_name': 'US_Equity_Fut_ChgP',
#                    'splitter': DataSplitter(test_size=25,
#                              initial_training_size=500,
#                              expanding=True,
#                              last_training_window=True),
#                    'target_preprocessing': make_pipeline(Difference(), Lag(-1)),
#                    'target_name': asset}



# scoring_list = [long_only_relative_mean_score, ctail_score, active_sharpe_score, active_mean_score, sharpe_score, mean_score]
# try1 = {'param_grid': {'func': [np.less, np.greater], 'thresh': np.linspace(-2, 2, num=20)},
#         'feature_preprocessing': make_pipeline(StandardScaler(), Clipper(-3, 3), StandardScaler())}

# try2 = {'param_grid': {'func': [np.less, np.greater], 'thresh': np.linspace(-0.05, 0.05, num=20)},
#         'feature_preprocessing': None}


# spx_change_dict_list = []
# for scoring in scoring_list:
#     for tr in [try1, try2]:
#         temp_dict = deepcopy(spx_change_dict)
#         temp_dict['scoring_func'] = scoring
#         temp_dict.update(tr)
#         spx_change_dict_list.append(temp_dict)
                    

In [ ]:
# # Original Config


# hyp = [('US_Equity_Fut_ChgP', '0', 'smaller', '1'),
#  ('US_Equity_Real_Vol1m_Chg', '0.1', 'mixed', '-3'),
#  ('US_Lois3m_Chg', '5', 'mixed', '-3'),
#  ('US_Equity_Vol_Chg', '0', 'larger', '1'),
#  ('Mom_Change_SPX', '0', 'larger', '1'),
#  ('DXY_ChgP', '0', 'smaller', '1'),
#  ('SPGSIN_Chg13', '0', 'larger', '1'),
#  ('eq_per_flows_USA', '0', 'smaller', '1'),
#  ('US_2y_v_OIS_ZSc52', '0', 'smaller', '1'),
#  ('US_Bond_30y_ZSc52', '0', 'smaller', '1'),
#  ('US_10y_2y_ZSc52', '0', 'smaller', '1'),
#  ('GLOB_EM_HCSp_Chg', '0', 'larger', '1')]



# def func_generator(cond):
#     if cond=='smaller':
#         return np.less
#     elif cond=='larger':
#         return np.greater
#     else:
#         return mixed
                

# meta_list = []
# for i in hyp:
#     meta_list.append(
#     {'feature_name': i[0],
#      'target_name': asset,
#      'param_grid': {
#          'func': [func_generator(i[2])],
#          'thresh': [float(i[1])]
#          }
#     }
#     )
    
# # for i in meta_list:
# #     i.update(common_meta_dict)

# Train/Test the SubStrats

In [ ]:
start_date = "2005"
SAVE = True  # This feature does NOT WORK yet. Can't pickle SubStrat unless we move into another file, but then Dask breaks...

if SAVE: 
    path = Path('logs', datetime.now().strftime('%d_%m_%Y_%H_%M_%S'))
    path.mkdir(parents=True)
    
for i, meta in enumerate(tqdm(substrat_meta_list)):
    
    print(meta)
    feature_name = meta['feature_name']
    target_name = meta['target_name']
    param_grid = meta['param_grid']
    splitter = meta['splitter']
    scoring_func = meta['scoring_func']
    feature_preprocessing = meta['feature_preprocessing']
    target_preprocessing = meta['target_preprocessing']

    features = FeatureFrame(df_master_weekly[[feature_name]].loc[start_date:].copy().dropna())

    target = FeatureFrame(df_master_weekly[[target_name]].loc[start_date:].copy().dropna()).squeeze()
    target = target_preprocessing.fit_transform(target)

    if feature_preprocessing is not None:
        estimator = deepcopy(feature_preprocessing)
        estimator.steps.append(('model', make_model(param_grid, scoring_func)))
    else:
        estimator = make_model(param_grid, scoring_func)

    wf = WindowFrame(
        features=features,
        target=target,
        splitter=splitter,
        estimator=estimator
    )

    wf.fit(parallel=True)
    
    # The magic line that solves all our pickling problems...
    SubStrat.__module__ = '__main__'
    
    if SAVE: 
        file_path = Path(path, f'window_frame_{i:03d}.h5')
        with open(file_path, "wb") as dill_file:
            dill.dump(meta, dill_file)
            dill.dump(wf, dill_file)
            
        meta_path = Path(path, f'window_frame_{i:03d}.json')
        with open(meta_path, 'w') as fp:
            json.dump({k: repr(v) for k, v in meta.items()}, fp, indent=4)
            
    # This next section is wrapped in a function so that we can load new 'wf'...
    # ... and process it without fitting again.
    def post_analysis(wf):
        signal = ThresholdSignSignal(wf)

        my_signals = signal.get_signals()
        optimal_signals = np.sign(wf.target.data).reindex(my_signals.index)

        strat_pnl = PNL(signal, asset_type=AssetType(level=False, log=True))

        buyhold_pnl = PNL(LongOnly(wf.target, my_signals.index), 
                          asset=wf.target.data,
                          asset_type=AssetType(level=False, log=True))

        fig, ax = plt.subplots(1, 1, figsize=(10, 3), dpi=100)
        strat_pnl.get_pnl().plot(label='Strategy')
        buyhold_pnl.get_pnl().plot(label='Buy&Hold')
        ax.legend()
        plt.tight_layout()
        plt.show()

        

        def return_stats(rets, name=None):
            print('\n')
            
            if name: print(name)
            print(f'Average Return: {rets.mean()}')
            print(f'Average Return Given A Signal: {rets[rets.abs()>0].mean()}')

            print(f'Volatility: {rets.std()}')
            print(f'Volatility Given A Signal: {rets[rets.abs()>0].std()}')

            print(f'Sharpe Ratio: {np.sqrt(52)*rets.mean()/rets.std()}')
            print(f'Sharpe Ratio Given A Signal: {np.sqrt(52)*rets[rets.abs()>0].mean()/rets[rets.abs()>0].std()}')

            print('\n')

        return_stats(buyhold_pnl.get_returns(), 'BUY&HOLD STRATEGY')
        return_stats(strat_pnl.get_returns(), 'NEW STRATEGY')

        def signal_stats(signals, name=None):
            if name: print(name)
            print(f'Long: {(signals>0).mean()} % of the time')
            print(f'Flat: {(signals==0).mean()} % of the time')
            print(f'Short: {(signals<0).mean()} % of the time')
            print('\n')

        signal_stats(optimal_signals, 'BUY&HOLD STRATEGY')
        signal_stats(my_signals, 'NEW STRATEGY')

        labels = [-1, 0, 1]
        cm = sk.metrics.confusion_matrix(optimal_signals.astype(int).values, my_signals.astype(int).values, labels=labels)
        sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=labels, yticklabels=labels)
        ax.set(xlabel='Predicted', ylabel='True')
        plt.tight_layout()
        plt.show()
        
    post_analysis(wf)

In [ ]:
# folder_path = Path('logs')
# folder_path = folder_path.d
folder_path = Path('logs', '01_12_2020_17_08_04')

meta_list, wf_list = [], []
for file_path in sorted(list(folder_path.glob('*.h5'))):
    with open(file_path, "rb") as dill_file:
        meta_list.append(dill.load(dill_file))
        wf_list.append(dill.load(dill_file))

In [ ]:
# for wf in wf_list:
#     post_analysis(wf)

In [ ]:
results = []
for wf in wf_list:
    signal = ThresholdSignSignal(wf)
    results.append(signal.signals)

In [ ]:
signals = pd.concat(results, axis=1)

In [ ]:
strats = signals.multiply(df_master_weekly['US_Equity_Fut'].diff(), axis=0).astype(float)

In [ ]:
strats.dropna().mean(axis=1).cumsum().plot(figsize=(10, 7))
df_master_weekly['US_Equity_Fut'].diff().reindex(strats.dropna().index).cumsum().plot()

In [ ]:
strats.columns = [i['feature_name']+str(n) for n,i in enumerate(substrat_meta_list)]

In [ ]:
signals.mean(axis=1).plot()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(11, 11), dpi=150)
sns.heatmap(strats.dropna().astype(float).corr(), cmap='Blues', ax=ax, annot=True, fmt='.2f')



In [ ]:
corrs = strats.astype(float).join(df_master_weekly[asset].diff()).corr()[asset].iloc[:-1]
weights  = 1/corrs.abs()
weights = weights/weights.sum()

(strats.dropna()*weights).sum(axis=1).cumsum().plot()
df_master_weekly['US_Equity_Fut'].diff().reindex_like(strats.dropna()).cumsum().plot()

In [ ]:
weights

In [ ]:
(strats.mean(axis=1)).cumsum().dropna().plot(legend=False)
df_master_weekly['US_Equity_Fut'].diff().reindex_like(strats.dropna()).cumsum().plot()